26.	The 3-digit number 376 in the decimal numbering system is an example of numbers with the special property that its square ends with the same digits: 3762 = 141376. Let's call a number with this property a steady square. Steady squares can also be observed in other numbering systems. In the base 14 numbering system, the 3-digit number c37 is also a steady square: c372 = aa0c37, and the sum of its digits is c+3+7=18 in the same numbering system. The letters a, b, c and d are used for the 10, 11, 12 and 13 digits respectively, in a manner similar to the hexadecimal numbering system. For 1 ≤ n ≤ 9, the sum of the digits of all the n-digit steady squares in the base 14 numbering system is 2d8 (582 decimal). Steady squares with leading 0's are not allowed. Find the sum of the digits of all the n-digit steady squares in the base 14 numbering system for 1 ≤ n ≤ 10000 (decimal) and give your answer in the base 14 system using lower case letters where necessary.

In [8]:
class AutomorphicNumberGenerator():
    def __init__(self):
        self.base = None
        self.digit_rep = '0123456789abcdefghijklmnopqrstuvwxyz'
        
    def generate(self, base, max_digit_count):
        assert(base in [10, 14])
        self.base = base
        
        all_digit_sums = 1 # Trivial cases: 0^0 = 0 and 1^2 = 1.
        for n in range(1, max_digit_count + 1):
            if n % 100 == 0:
                print('Current num of digit (every 100):', n)
            for num in self._generate_n_digit(n):        
                digit_sum = self._get_digit_sum(num)
                all_digit_sums += digit_sum
        print('Digit sum:', self._to_literal(all_digit_sums))
        
    def _generate_n_digit(self, n):
        rv = set()
        a, b = (self.base//2)**n, 2**n
        x, y = self._extended_gcd(a, b)
        lower_bound = self.base**(n-1)
        k = a*(x % b)
        if k >= lower_bound:
            rv.add(k)
        k = a*((-x) % b) + 1
        if k >= lower_bound:
            rv.add(k)
        return rv
    
    def _extended_gcd(self, a, b):
        (x, y) = (0, 1)
        (last_x, last_y) = (1, 0)
        while b != 0:
            (q, r) = divmod(a, b);
            (a, b) = (b, r)
            (x, last_x) = (last_x - q * x, x)
            (y, last_y) = (last_y - q * y, y)
        return (last_x, last_y)
    
    def _get_digit_sum(self, num):
        rv = 0
        q = num
        while q:
            q, r = divmod(q, self.base)
            rv += r
        return rv
    
    def _to_literal(self, num):
        rv_backtrace = ''
        q = num
        while q:
            q, r = divmod(q, self.base)
            rv_backtrace += self.digit_rep[r]
        return rv_backtrace[::-1]
    
def main():
    g = AutomorphicNumberGenerator()
    g.generate(14, 10000)
    
print("Answer :")
main()

Answer :
Current num of digit (every 100): 100
Current num of digit (every 100): 200
Current num of digit (every 100): 300
Current num of digit (every 100): 400
Current num of digit (every 100): 500
Current num of digit (every 100): 600
Current num of digit (every 100): 700
Current num of digit (every 100): 800
Current num of digit (every 100): 900
Current num of digit (every 100): 1000
Current num of digit (every 100): 1100
Current num of digit (every 100): 1200
Current num of digit (every 100): 1300
Current num of digit (every 100): 1400
Current num of digit (every 100): 1500
Current num of digit (every 100): 1600
Current num of digit (every 100): 1700
Current num of digit (every 100): 1800
Current num of digit (every 100): 1900
Current num of digit (every 100): 2000
Current num of digit (every 100): 2100
Current num of digit (every 100): 2200
Current num of digit (every 100): 2300
Current num of digit (every 100): 2400
Current num of digit (every 100): 2500
Current num of digit (eve

30.	For fixed integers a, b, c, define the crazy function F(n) as follows: F(n) = n - c for all n > b F(n) = F(a + F(a + F(a + F(a + n)))) for all n ≤ b. Also, define S(a,b,c)=∑n=0bF(n). For example, if a = 50, b = 2000 and c = 40, then F(0) = 3240 and F(2000) = 2040. Also, S(50, 2000, 40) = 5204240. Find the last 9 digits of S(217, 721, 127).

In [7]:
class CrazyFunction():
    def get(self, n, a, b, c):
        if n > b:
            return n - c
        delta = 1 - 4*a + 3*c
        delta_pos, in_delta_pos = divmod(b - n, a) 
        return b - c + 1 - (delta_pos + 1) * delta - in_delta_pos - (a - 1) * delta_pos

    def get_sum(self, a, b, c):
        good_part_count = (b + 1)//a
        good_part_delta = self.get(b - a, a, b, c) - self.get(b, a, b, c)

        good_begin_pos = b - a + 1
        good_end_pos = b
        good_begin_value = self.get(good_begin_pos, a, b, c)
        good_end_value = self.get(good_end_pos, a, b, c)
        good_part_sum = (good_begin_value + good_end_value) * a//2
        good_sum = (2 * good_part_sum + good_part_delta * (good_part_count - 1) * a) * good_part_count//2

        rest_begin_pos = 0
        rest_end_pos = b - a * good_part_count
        rest_begin_value = self.get(rest_begin_pos, a, b, c)
        rest_end_value = self.get(rest_end_pos, a, b, c)
        rest_sum = (rest_begin_value + rest_end_value) * (rest_end_pos - rest_begin_pos + 1)//2

        return rest_sum + good_sum

class Problem():
    def solve(self):
        self.sanity_check()
        self.get()

    def get(self):
        function = CrazyFunction()
        result = function.get_sum(21**7, 7**21, 12**7)
        print(result, result % 10**9)

    def sanity_check(self):
        function = CrazyFunction()
        assert(function.get(0, 50, 2000, 40) == 3240)
        assert(function.get(2000, 50, 2000, 40) == 2040)
        assert(function.get_sum(50, 2000, 40) == 5204240)

def main():
    problem = Problem()
    problem.solve()

print("Answer :")
main()

Answer :
770623859492181794104378494291504964 291504964
